# Examples for Chapter 9

In [1]:
%matplotlib inline

## Sequential SAR change detection on GEE

In [2]:
import ee, math, time
from ipyleaflet import (Map,DrawControl,TileLayer,basemaps)
from auxil.eeWishart import omnibus
ee.Initialize()

### Auxiliary functions

In [3]:
def get_vvvh(image):   
    ''' get 'VV' and 'VH' bands from sentinel-1 imageCollection and restore linear signal from db-values '''
    return image.select('VV','VH').multiply(ee.Image.constant(math.log(10.0)/10.0)).exp()

def get_image(current,image):
    ''' accumulate a single image from a collection of images '''
    return ee.Image.cat(ee.Image(image),current)    
    
def clipList(current,prev):
    ''' clip a list of images '''
    imlist = ee.List(ee.Dictionary(prev).get('imlist'))
    poly = ee.Dictionary(prev).get('poly')    
    imlist = imlist.add(ee.Image(current).clip(poly))
    return ee.Dictionary({'imlist':imlist,'poly':poly})

def handle_draw(self, action, geo_json):
    global poly
    if action == 'created':
        coords =  geo_json['geometry']['coordinates']
        poly = ee.Geometry.Polygon(coords)
        
dc = DrawControl()
dc.on_draw(handle_draw)

def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

### Display the map

In [17]:
m = Map(center=[50.9, 6.4], zoom=11, basemap=basemaps.OpenStreetMap.HOT,layout={'height':'500px'})
m.add_control(dc)

m

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cDovL3tzfS50aWxlLm9wZW5zdHJlZXRtYXAuZnIvaG90L3t6fS97eH0ve3l9LnBuZycsICdhdHRyaWJ1dGlvbic6ICcmY29weTsgPGEgaHJlZj0iaHTigKY=


### Run the algorithm

In [11]:
try:
    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
              .filterBounds(poly) \
              .filterDate(ee.Date('2017-04-01'), ee.Date('2017-11-01')) \
              .filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV','VH'])) \
              .filter(ee.Filter.eq('resolution_meters', 10)) \
              .filter(ee.Filter.eq('instrumentMode', 'IW')) 
    collection = collection.sort('system:time_start') 
# comment out on first run to determine relative orbit numbers ******************
    collection = collection.filter(ee.Filter.eq('relativeOrbitNumber_start',139)) 
# *******************************************************************************
    acquisition_times = ee.List(collection.aggregate_array('system:time_start')).getInfo()
    count = len(acquisition_times) 
    if count<2:
        raise ValueError('Less than 2 images found')
    timestamplist = []
    for timestamp in acquisition_times:
        tmp = time.gmtime(int(timestamp)/1000)
        timestamplist.append(time.strftime('%x', tmp))  
    #          make timestamps in YYYYMMDD format            
    timestamplist = [x.replace('/','') for x in timestamplist]  
    timestamplist = ['T20'+x[4:]+x[0:4] for x in timestamplist]
    relativeorbitnumbers = str(map(int,ee.List(collection.aggregate_array('relativeOrbitNumber_start')).getInfo()))
    print 'Images found: %i'%count
    print 'Acquistion dates'
    print timestamplist
    print 'Relative orbit numbers'
    print relativeorbitnumbers
    pcollection = collection.map(get_vvvh)
    pList = pcollection.toList(100)   
    first = ee.Dictionary({'imlist':ee.List([]),'poly':poly}) 
    imList = ee.Dictionary(pList.iterate(clipList,first)).get('imlist')
    result = ee.Dictionary(omnibus(imList,0.0001,True))
except Exception as e:
    print 'An error occurred in Omnibus: %s'%e


Images found: 35
['T20170402', 'T20170408', 'T20170414', 'T20170420', 'T20170426', 'T20170502', 'T20170508', 'T20170514', 'T20170520', 'T20170526', 'T20170601', 'T20170607', 'T20170613', 'T20170619', 'T20170625', 'T20170701', 'T20170713', 'T20170719', 'T20170725', 'T20170731', 'T20170806', 'T20170812', 'T20170818', 'T20170824', 'T20170830', 'T20170905', 'T20170911', 'T20170917', 'T20170923', 'T20170929', 'T20171005', 'T20171011', 'T20171017', 'T20171023', 'T20171029']
[139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139]


### Display results

In [12]:
jet = 'black,blue,cyan,yellow,red'

cmap = ee.Image(result.get('cmap')).byte()   
smap = ee.Image(result.get('smap')).byte()
fmap = ee.Image(result.get('fmap')).byte()  
bmap = ee.Image(result.get('bmap')).byte()

m.add_layer(
    TileLayer(url=GetTileLayerUrl(
        fmap.visualize(min=0, max=count/2, palette=jet,opacity = 0.4)
    )
))

### Export to GEE assets